In [2]:
import urllib.parse
from selenium import webdriver
import time
import random
from bs4 import BeautifulSoup
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np

In [3]:
def move_page(searching, page):
    base_url = 'https://section.cafe.naver.com/ca-fe/home/search/articles'
    query_params = {
        'q': searching,
        't': '1710744519825',
        'em': '1',
        'p': page
    }
    url = f'{base_url}?{urllib.parse.urlencode(query_params)}'
    return url

def make_cafe_list(driver):
    cafe_list = []
    html_page = driver.page_source
    soup_page = BeautifulSoup(html_page, 'html.parser')
    div_elements = soup_page.find_all('div', class_='article_item_wrap')
    for div_element in div_elements:
        a_tags = div_element.find_all('a', target='_blank')
        for a_tag in a_tags:
            # a 태그의 class가 'cafe_info'인 경우는 건너뛰기
            if 'cafe_info' in a_tag.get('class', []):
                continue
            href = a_tag.get('href')
            cafe_list.append(href)
    return cafe_list


In [4]:
# 게시물 정보 가져오기 함수
def get_data(driver):
    driver.switch_to.frame('cafe_main')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    reply_list = []
    # 제목
    title = soup.select('h3.title_text')[0].text.strip()
    
    # 작성일
    date= soup.select('span.date')[0].text
    date_match = re.search(r'\d{4}\.\d{2}\.\d{2}', date)
    date = date_match.group() if date_match else None
    
    # 본문
    try:
        content = soup.select('div.se-main-container')[0].text
        content = re.sub(r'\n+', '', content)
        # content = re.sub(r'\u200b+', '', content)
        content = content.replace('\u200b', ' ')
        
    except:
        pass
    
    # 댓글
    try:
        all_reply = soup.select('span.text_comment')
        for reply in all_reply:
            reply = reply.text.strip()
            reply = re.sub(r'\n', '', reply)
            reply_list.append(reply)
    except:
        pass
    
    data = [title, date, content, reply_list]
    return data
    

In [8]:
page = range(1,3)

In [9]:
data_list = []
for p in page:
    driver = webdriver.Chrome()
    url = move_page('망원시장', p)
    driver.get(url)
    
    # 해당 페이지의 카페 리스트 생성
    cafe_list = make_cafe_list(driver)
    
    # 각 카페에 접속하여 데이터 추출
    for index, cafe_href in enumerate(cafe_list):
        # 새로운 탭에서 카페 접속
        driver.execute_script("window.open('{}', '_blank');".format(cafe_href))
        time.sleep(3)
        # 새로 열린 탭으로 전환
        new_window = driver.window_handles[-1]
        driver.switch_to.window(new_window)
        try:
            data = get_data(driver)
            data_list.append(data)
        except:
            pass
        finally:
            # 새 탭 닫기
            driver.close()
            # 메인 탭으로 전환
            main_window = driver.window_handles[0]
            driver.switch_to.window(main_window)
        
        # 마지막 카페인 경우 더 이상 링크 접속하지 않음
        if index == len(cafe_list) - 1:
            break

# 작업이 끝나면 드라이버를 종료
driver.quit()

In [10]:
data_list

[['망원시장 다녀왔어요~',
  '2024.03.04',
  ' 안녕하세요~~ 지난주에 다녀온 망원시장 맛집인 우이락에 대해 소개해볼까 합니다! 웨이팅도 길고 줄도 길고 날씨도 무진장 추웠어요 ㅠㅠ 하지만 굴하지 않고 원격 줄서기를 이용하여 열심히 대기하여 입장했답니다 ^^ 크림막걸리까지 영수증 리뷰 이벤트를 하여 맛있게 냠냠 했어요~~ 고추튀김이 크면 얼마나 클 것이며 맛있으면 얼마나 맛있겠어?? 라고 별스럽지 않게 생각했는데 실제로 먹어보니 왜 사람들이 이 추운날 줄까지 서가며 먹는지 이유를 알겠더라고요 ^0^ 바삭하고 속은 꽉찬것이 구미가 당기는 맛이었어요~ 여러분들도 망원시장 가실 일 있으시면 꼭 찾아가서 드셔보시길 적극!! 추천드립니다 +_+ ',
  ['오! 여기 늘 사람들이 많이 줄 서 있어서 먹어볼 생각을 못했는데! 다음에 가면 꼭 먹어봐야겠어요!!🤤',
   '원격줄서기 하시고 다른곳 가셨다 가시는걸 추천드려요~',
   '여기 항상 줄이 길어서 못먹어봤는데 맛있다니 더 궁금하네요 ㅋ',
   '꼭 방문해보세요~ 강추드리옵니다 ㅎㅎ',
   '망원시장 맛집 너무 많죠~~~',
   '맞아요 저는 개인적으로 광장시장보다 훨 좋았어요~',
   '꼭한번 가보구싶어요',
   '추천추천드립니다 ^^']],
 ['망원시장 닭강정',
  '2024.02.28',
  '망원시장 다녀왔어요 큐스 닭강정 맛집이라서 포장해 왔어요큐스 닭강정은 맛종류가 많아서 한참 고민했어요.오리지널 간장 뿌링클 3가지 맛으로 한마리 주문했어요.이 위에 떡도 올려주셨답니다. 집에 와서 별 기대없이 먹었는데 역시 맛집이네요. 식었어도 바삭한 식감에 달짝지근한 양념맛 입맛에 딱 맞아서 나중에 또 사먹으려고요! 종류가 다양하니 다음에 맛별로 먹어보고  싶어용 ~~    ',
  ['츄릅츄릅.. 맛있어보여요 😲', '침고여요 ㅎㅎ', '찜 해놓을게요', '망원시장 유명하더라고요 😀']],
 ['망원 시장에서 이쁜 그래비티 회원님 포착!!',
  '2024.03.05',
  '탐나는레

In [11]:
df1 = pd.DataFrame(data_list, columns = ['title', 'date', 'content', 'reply'])
df1

,title,date,content,reply
0,망원시장 다녀왔어요~,2024.03.04,안녕하세요~~ 지난주에 다녀온 망원시장 맛집인 우이락에 대해 소개해볼까 합니다! ...,[오! 여기 늘 사람들이 많이 줄 서 있어서 먹어볼 생각을 못했는데! 다음에 가면 ...
1,망원시장 닭강정,2024.02.28,망원시장 다녀왔어요 큐스 닭강정 맛집이라서 포장해 왔어요큐스 닭강정은 맛종류가 많아...,"[츄릅츄릅.. 맛있어보여요 😲, 침고여요 ㅎㅎ, 찜 해놓을게요, 망원시장 유명하더라..."
2,망원 시장에서 이쁜 그래비티 회원님 포착!!,2024.03.05,탐나는레이 회원목격담 게시판 글+사진 작성시! 꼭 [사진의 번호판]을 가린후 사진...,[경기그래그래님이신거 같은데... 클럽 스티커는 다떼시고 넘버링만 남기셨네유? 중고...
3,망원시장에~~,2024.02.08,이번주에 설명절이 있어서 장보러 망원시장 왔지요~~어마어마하게 사람 뚫고 다니며 장...,"[저도 제기시장갔다왔는데 사람들이 어마어마해요~~~, 내일은 더 많을거 같아요;;..."
4,망원시장 송이네 분식,2024.01.07,서민갑부에도 출연했던망원시장 송이네 3평 매장에서도 엄청나게 잘되어서많은 부러움을...,"[대파김밥맛이 궁금하던데 맛있다시니. 한번 가보고싶네요^^, 시장에서 사먹는 분식은..."
5,오랜만에 파스타 점심^^(망원시장표 백합),2024.02.27,어제 두찌가 먹고 싶다는 딸기 사러 망원 시장엘 갔는데 오. 거의다 한팩에 만원대라...,[앗 저도 주말에 망원시장가서 5천원 생백합 한바구니 사와서 시원하게 백합탕 끓여먹...
6,망원시장 투어,2024.01.24,작년 말에 티비에서 망원시장이 나왔어요. 집에서도 가깝고 재래시장 돌아다니는 것도 ...,"[망원시장은 안가봤는데 시간나면 한번 가봐야겠어요 맛있는 음식들이 많네요~^^, 네..."
7,망원시장 딸기시세 어떻게 되나요?,2024.01.05,딸기가 여전히 비싸네요ㅜㅜ망원시장은 그나마 좀 싼것 같은데시세가 어떻게 되나요?,"[어제 지나는길에 쓱 봤을땐 크기가 큰건 만원정도였고, 만원 이하짜리도 있었어요~ ..."
8,망원시장 맛집 밥집 한정식 추천 정보와 지인/직원 할인 아시는분?,2024.03.05,스마트하게 살고 계신 카페 여러분들 잘 계신가요?이번에는 망원시장 맛집 밥집 한정식...,[맛집 찾는분 은근 계신것 같더라고요 망원시장 거주 중이신 회원님들께 여쭤보시면 될...
9,망원시장 맛집은 튀김이랑 떡갈비에요? 또 뭐있나요,2024.02.25,망원시장 데이트 가려고요!맛집 알아보는데 거기에 있는데 튀김집이랑 떡갈비 가게 글이...,"[망원시장 반찬도 맛있어요, 꽈배긴가 뭐 유명했던거 같은데, 시장 바깥이긴하지만 소..."


In [5]:
total_data_list = []

In [6]:
data_list = []
driver = webdriver.Chrome()
p = 61
url = move_page('자양시장', p)
driver.get(url)
time.sleep(3)
# 해당 페이지의 카페 리스트 생성
cafe_list = make_cafe_list(driver)

# 각 카페에 접속하여 데이터 추출
for index, cafe_href in enumerate(cafe_list):
    # 새로운 탭에서 카페 접속
    driver.execute_script("window.open('{}', '_blank');".format(cafe_href))
    time.sleep(3)
    # 새로 열린 탭으로 전환
    new_window = driver.window_handles[-1]
    driver.switch_to.window(new_window)
    try:
        data = get_data(driver)
        data_list.append(data)
        time.sleep(1)
    except:
        pass
    finally:
        # 새 탭 닫기
        driver.close()
        # 메인 탭으로 전환
        main_window = driver.window_handles[0]
        driver.switch_to.window(main_window)
        time.sleep(1)

# 작업이 끝나면 드라이버를 종료
driver.quit()


In [7]:
data_list

[['[내마반 30기 7년안에 100억대 부자될 4람들조 미니파이]4주차 과제',
  '2023.02.15',
  '부동산 연락하기 어디로 방문해야 할지 몰라머리와 마음이 복잡스러웠지만,가장 현실적으로 접근!현재 살고 있는 동네에서 아이들 교육 문제로 많이이사가는 자양동과 구의동, 광장동 중자양동으로 결정 후 네이버 부동산에서 가장 인상 좋은 부사님이 계신 부동산으로 전화 다음 날 매물을 보기로 하고 약속을 잡았습니다.미리 문자로 몇개의 매물과 함께편한 운동화 신고 오라고 남겨주셨습니다.  부동산 방문하기 부동산 예약 시간 전에 미리 도착해서 주변을 미리 둘러봐야지 했지만생각보다 시간이 넉넉하지 않아 바로 부동산으로 입장 너무나도 좋은 인상의 부사님이컬러출력된 전체 단지들과 방문할 매물들의 위치를 미리 체크해서주셨습니다. (너무 보이게 들고 다니진 말아달라는 주의사항과 함께)     매물 보기 총 4개의 물건을 보여 주셨습니다.2개의 매물은 다른 부사님과 접선해서 보기도 했습니다. 사실 막상 들어가니 어떻게 해야 할지 몰라 물은 잘나오죠??누수는 없나요??AI처럼 몇마디 못하고 나왔습니다. 또 부사님이 임대사업자 물건이라고 보여주시는데너나위님 강의 내용이 생각나서괜히 반가운 마음도 들었습니다. 걷는 것을 좋아해 집보며 걷는 것이  어렵지 않을 것 같았는데생각보다 힘들었습니다.집에와서 방전 ㅠ  느낀점 자양동은 구축아파트가 많은 편인데 초, 중,고를 둘러싸고 아파트 단지 들이 형성 되어있습니다.왠지 건대의 복잡한 분위기나 양꼬치 거리를 생각해 거주 환경이 별로이지 않을까라는 편견이 있었습니다. 직접 매물을 보니 거주지로서의 자양동은건대역이 가깝고 건대 롯데백화점과 이마트가 있고걸어서 한강을 가기에도 편리한 곳입니다. 건대 뒤쪽으로 자양동은 한적하고 학원가도 형성되어 있으며,자양골목시장도 있어 생활하기 굉장히 편리한 곳이었습니다. 직접 가보니 눈에 보이는 것들이 많았습니다.부사님은 자양동은 가격대가 많이 내려가는 편이 아니라고 하시며주재원으로 오셔서 사시는 분도 

In [8]:
len(data_list)

3

In [670]:
total_data_list.extend(data_list)

In [671]:
total_data_list

[['구의역도보5분/ 21평/방3개,화1/빌라 매매/전세가능 도보5분',
  '2022.05.20',
  "※ 게시글 제목은 [지역 or 근처 지하철역+매물유형+거래유형+매물가격]으로 필수 기재 (위반 시 게시글 삭제 및 제재) ※ 말머리 필수 사용 (위반 시 게시글 삭제 및 제재) ※ 게시글 양식 삭제/임의 변경 불가! (위반 시 제재) ※ 반드시 '직거래'를 희망하는 매물만 등록 가능! ※ 부동산 업체 매물 등록 불가! (위반 시 제재)   ▶ 빠른 공실 해결을 위해 부동산의 도움을 받길 희망하시나요? (Y / N) :  Y▶ 피터팬 APP, WEB에도 함께 노출하시겠습니까? (Y / N) :  Y  ▶ 주소 :  광진구 자양동 218-30 신성빌라101호▶ 건물형태 :  빌라▶ 거래 유형 :  매매/전세가능▶ 가격 :  5억8천/4억▶ 방개수 :  3개/화장실1개▶ 옵션정보 :  빌트인 냉장고,식세기,에어컨(매매시)▶ 관리비 :  1만5천원▶ 관리비 포함내역 :  계단청소 및 건물관리▶ 층수 :  1층/4층▶ 면적 :  79제곱/21평▶ 난방 방식 :  도시가스▶ 주차 가능 :  가구당1대▶ 이사가능일 :  9월초▶ 문의 연락처 :  010.2385.6616  ▶ 상세 설명 : 올 리모델링 한지 6개월도 안되었습니다자양시장이 옆에 있어 너무좋구요2호선라인 출퇴근도 너무 좋습니다.건대도 가깝고어린이대공원도 가깝습니다.층간소음 걱정없이 아이들 뛰어놀기 너무좋아요  ▶ 매물 사진 :  ",
  []],
 ['내 마음을 뺏어 봐~! 🍾(SM구판장)',
  '2023.01.11',
  '작년 말에 "굿바이, 구판장~!" 하고 인사했지만,이제 새해가 되었으니 "하이, 헬로, 구판장~!" 입니다. 😅 우리 동네 시장보다 더 자주 가는 자양 전통 시장.시장 길목에 들어서면 걸음도 빨라집니다.그녀를 만나는 곳 100미터 전처럼. 💓그리고 사랑하는 그들을 만났습니다!내 사랑~!🍾🍾🍾🍾  12월 한달 걸렀더니, 신상이 꽤 들어왔네요.그 사이 조용히 들어왔다, 바람과 함께 사

In [672]:
len(total_data_list)

41

In [673]:
df61_60 = pd.DataFrame(total_data_list, columns = ['title', 'date', 'content', 'reply'])

In [674]:
df51_60.to_csv('naver_j_51_60.csv', index=False)

In [675]:
pd.read_csv('naver_j_51_60.csv')

,title,date,content,reply
0,"구의역도보5분/ 21평/방3개,화1/빌라 매매/전세가능 도보5분",2022.05.20,※ 게시글 제목은 [지역 or 근처 지하철역+매물유형+거래유형+매물가격]으로 필수 ...,[]
1,내 마음을 뺏어 봐~! 🍾(SM구판장),2023.01.11,"작년 말에 ""굿바이, 구판장~!"" 하고 인사했지만,이제 새해가 되었으니 ""하이, 헬...","['오 샴페인 네병보기만 해도 흐뭇하네요', '너무 아름답습니다!🥰', '프레드릭 ..."
2,국밥충 출근 완료우~ 자양동 - 조현우국밥,2021.07.10,"자양동 주민이고, 국밥충이라면 이 곳을 모를리가 없지요우~ 배달의 민족 검색해보면...","['조현우 너 딱 기다려라.♡', '이 포스팅을 영즈님 헌정 포스팅으로 쓰려고 했었..."
3,광진구커튼 자양동 베이지 쉬폰커튼 우드블라인드 방문제작설치,2024.02.17,광진구커튼 자양동 베이지 쉬폰커튼 우드블라인드 방문제작설치 안녕하세요...,[]
4,자양4동 한강변 지주분들 단톡방입니다,2021.04.02,지도에표시한 네모부분안에 소유주이신분들 모십니다. 지주택사업은 배제하고 재개발에 관...,"['옆동네 아크로포레스트나 트리마제처럼 개발되는거 보고 싶네요', '현 부동산 도정..."
5,광진구 자양동커튼 베이지 암막커튼 나비주름 콤비블라인드 맞춤제작후기,2024.02.25,광진구 자양동커튼 베이지 암막커튼 나비주름 콤비블라인드 맞춤제작후기 안녕하세...,[]
6,서울 와인 성지 추천합니다,2020.12.30,요즘 집콕이 대세다 보니 눈치보여 친구랑 술한잔 하기도 힘든 시절입니다.그래서 집에...,"['저두 라빈 이용하는데, 새마을 구판장이라...거기 가봐야 겠네요 ^^', '네 ..."
7,"동서울 양평해장국, 자양시장 혜성순대국",2022.04.08,자양사거리 쪽 돼지갈비집이 양평해장국 집으로 바뀌었길래 궁금해서 가봤어요😁이전 돼지...,"['ㅋㅋㅋ마지막 닭발 사진 보고내용 읽고는 빵 터졌네요', 'ㅋㅋㅋ진짜 안마시려고했..."
8,자양동 호남아구찜 - 선도좋은 알탕과 곤이,2022.04.26,자양골목시장이 있는 혜민병원의 부근에 위치한 식당이랍니다.지나가다 보면 동네분이 자...,"['가보고 싶네요', '궁금했는데 정보 감사합니다.', '알탕 급 땡기네요~^^;'..."
9,건대 트레비앙 서울 광진구 화양동 오피스텔 즉시입주 분양가24시,2024.03.12,건대 트레비앙 서울 광진구 화양동 오피스텔 즉시입주 분양가24시건대 트레비앙 모델하...,['건대 트레비앙 서울 광진구 화양동 오피스텔 즉시 입주 분양가는 얼마정도인가요?'...


In [676]:
naver_j_1_10 = pd.read_csv('naver_j_1_10.csv')
naver_j_11_30 = pd.read_csv('naver_j_11_30.csv')
naver_j_31_50 = pd.read_csv('naver_j_31_50.csv')
naver_j_51_60 = pd.read_csv('naver_j_51_60.csv')

In [682]:
jayang_cafe = pd.concat([naver_j_1_10, naver_j_11_30, naver_j_31_50, naver_j_51_60], axis=0)
jayang_cafe.to_csv('jayang_cafe.csv', index=False)

In [678]:
naver_1_10 = pd.read_csv('naver_1_10.csv')
naver_11_20 = pd.read_csv('naver_11_20.csv')
naver_21_30 = pd.read_csv('naver_21_30.csv')
naver_31_40 = pd.read_csv('naver_31_40.csv')
naver_41_50 = pd.read_csv('naver_41_50.csv')

In [684]:
mangwon_cafe = pd.concat([naver_1_10, naver_11_20, naver_21_30, naver_31_40, naver_41_50], axis=0)
mangwon_cafe.to_csv('mangwon_cafe.csv', index=False)

In [685]:
jayang_cafe

,title,date,content,reply
0,구의역 5분 이내 자양시장 근처(계약완료),2024.03.10,👇고시원 원장님이라면 꼭 보세요! https://bit.ly/4bVnzmV 🆘 빠른...,[]
1,[마감] 12/31(일) 자양시장 낮벙(23년 마지막 벙개?),2023.12.30,■ 회칙 준수 : 처음 모임에 나오는 회원님은 https://cafe.nave...,"['참석1 온종일', '너구리 참', '와우~ 감사합니닷🫢참석2', '사월 참', ..."
2,자양시장 과일집 추천이요~,2024.01.04,얼마전에 이사를 왔는데요자양시장 몇번 다니면서 정육점은 이제 찾았는데(제 입맛에 맞...,['과일집 찾으실 때까지 마켓컬리 어떠세요 마켓컬리로 시키면 당일 밤 아니면 바로 ...
3,자양시장 맛집 어디있나요?,2023.07.31,자양시장에가면 꼭 들른다 하는곳 있나요?이사와서 어딜가야할지..^^;;몸두무겁고 덥...,['자양전통시장 말씀하시는걸까요 ?? ㅎㅎ저는 시장가면착한탕국 : 국 끓여둔 거 포...
4,새마을 구판장에서 와인사고 자양 시장에서 안주도 사고...,2023.03.18,유바비 님이 올려주신 새마을 구판장 이번주 행사 보고 처음으로 새마을 구판장에 가서...,['탄닌 세지 않으면서 부드럽게 술술 마시기 딱 좋더라구요. 괜히 신의 물방울에 나...
...,...,...,...,...
36,"[2030] 설마님과 함께, 1조벌자 온라인 OT (완전체 만남👏)",2024.01.17,안녕하세요!2030 리치해빗 2기 1조 조장 준언서입니다. 저희 조는 아이스연유라떼...,"['앗!! 분신술~ ㅎㅎ', 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 저희도 깜짝 놀랐어요!!', '..."
37,(계약완료)광진구 자양동 투룸2000-40,2020.12.15,※ 게시글 양식을 삭제/임의 변경 하지 마시고 정보만 입력해주세요. ▶ 피터팬 ...,[]
38,광진 kt부지에 아파트 들어서고나면,2022.06.15,광진 kt부지에 들어서는 아파트가 2024년 말? 완공이라고 들었는데교통대책(도로확...,['여기 분양했나요?']
39,구의역 건대입구역 도보10분 반지층 투룸월세 350/30 ***(계약완료),2020.11.24,매물 상세정보 매물주소광진구 자양동 22*-5호매물가격보증금 / 월세보증금350/...,[]


In [686]:
mangwon_cafe

,title,date,content,reply
0,망원시장,2024.03.09,신촌세브란스 가서 검사하고 5.8키로에있는망원시장 갔는데요 길하나 사이로 월드컵 시...,"['사람이 막 많진 않네요?', '오전 10시 넘어서라 한산했어요~^^ 그래서 편..."
1,망원시장 투어,2024.03.01,"망원동에 볼 일 본 다음 망원시장 투어하고, 홍두깨 칼국수, 수제비 5천원에 먹고 ...","['여기 칼국수랑 수제비 넘 맛나죠 사진 보니까 저도 가서 먹고싶네요 ㅎㅎ', '네..."
2,망원시장,2024.02.26,서대문형무소 역사관에서 관람하고나오니 배에서밥달라고 아우성이라 어디 갈까 찾아보던중...,"['우와! 순대, 튀김, 떡볶이 호떡… 다 맛나보여요 ㅠㅠ 배고파요 ㅠㅠ', '식사..."
3,망원시장 호다닥,2024.03.04,일단 시장세권 거주민분들 부러워요 주말 금지 ~아침 11시 이후 혼잡.. 주차장 ...,['ㅎㅎㅎ바쁘다 바빠 꼬띠엄마^^ 늘 부지런히 집밥 해먹는 모습이 존경스러워요 집밥...
4,망원시장 명물이죠~,2024.03.02,딸기아이스크림+마시멜로우 로스팅 해주는데요달달구리 아주 맛있어요~망원시장은 복잡하지...,"['전 거기 칼국수 조아해용ㅋ', '망원시장 여러번 갔지만 칼국수는 안먹어봤는데 찾..."
...,...,...,...,...
54,"홍대 합정 연남 망원 독립오피스 사무실임대, 보증금 146만원 월세73만원 ★계약완료﻿★",2024.01.08,📋 매물 등록자 정보 🔹﻿세입자 / 임대인: 임대인🔹연락처: 010 8243 13...,[]
55,점심은 망원돈까스~,2023.06.15,오늘은 점심으로 엄마랑 여동생이랑 세모녀가 늦은 점심 먹으러 망원돈까스가서 모밀이랑...,"['오 !저도 소스에 담궈져있는 돈까스 좋아해용 ㅎ', '옛날 왕돈까스 스타일이 좋..."
56,망원 시장 칼 갈아주는곳?,2021.02.18,망원시장 칼 갈아 주는곳 있나요?위치 좀 자세히 알려주세요 ~~~,"['저도궁금하네요', '시장이라 있을것같긴한데..']"
57,망원시장에 가요,2022.12.11,어디갈까 하다가 망원시장에 갔어요먹거리 볼거리 풍성?줄서서 한참 기다려서야 먹을수 ...,['동교동 살때 몇번 갔었는데 아쉬워요.한번 가면 장볼겸 두세시간은 놀다오곤 했죠....
